In [1]:
import sys
sys.path.append('../utils')

In [2]:
import pipeline
import importlib
import pandas as pd

importlib.reload(pipeline)
from pipeline import extract_session_data, clean_session_data, get_driver_laps, get_manual_finish_dict
from pipeline import assemble_race_dataset, extract_fp2_features, extract_quali_features, assemble_race_dataset_pre_race
from fastf1 import get_session

In [3]:
# Singapore GP 2025 (Round 18)
# This cell was made before the race so it excludes the race finish data 
df_fp2_18 = extract_session_data(2025, "Azerbaijan", "FP2")
df_quali_18 = extract_session_data(2025, "Azerbaijan", "Q")

fp2_clean_18 = clean_session_data(df_fp2_18)
quali_clean_18 = clean_session_data(df_quali_18)

fp2_features_18 = extract_fp2_features(fp2_clean_18)

quali_features_18 = extract_quali_features(quali_clean_18)
                       
race_df_18 = assemble_race_dataset_pre_race(fp2_features_18, quali_features_18)
race_df_18["grand_prix"] = "Singapore"
race_df_18

req         WARNING 	DEFAULT CACHE ENABLED! (4.29 GB) /Users/illiapolishchuk/Library/Caches/fastf1
core           INFO 	Loading data for Azerbaijan Grand Prix - Practice 2 [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core     

,Driver,fp2_avg_lap,fp2_best_lap,fp2_total_laps,FastestQualiLap,QualiPosition,grand_prix
0,ALB,109.180286,102.177,14,103.778,20,Singapore
1,ALO,119.446529,102.967,17,101.857,12,Singapore
2,ANT,109.124000,101.779,10,101.464,6,Singapore
3,BEA,117.111133,101.891,15,102.666,15,Singapore
4,BOR,110.747818,102.561,11,102.277,14,Singapore
5,COL,106.806250,103.322,12,102.779,16,Singapore
6,GAS,119.708308,102.674,13,103.139,19,Singapore
7,HAD,108.606538,102.443,13,101.647,9,Singapore
8,HAM,107.718333,101.293,9,101.821,11,Singapore
9,HUL,105.979333,102.820,12,102.916,17,Singapore


In [4]:
race_df_18.to_csv("../data/r18_singapore_2025.csv", index=False)

In [5]:
import joblib

model  = joblib.load("../models/logreg_scaled_quali_pre_r18_singapore.pkl")
scaler = joblib.load("../models/scaler_pre_r18_singapore.pkl")

In [6]:
import pandas as pd

df_singapore = pd.read_csv('../data/r18_singapore_2025.csv')
df_singapore.head()

,Driver,fp2_avg_lap,fp2_best_lap,fp2_total_laps,FastestQualiLap,QualiPosition,grand_prix
0,ALB,109.180286,102.177,14,103.778,20,Singapore
1,ALO,119.446529,102.967,17,101.857,12,Singapore
2,ANT,109.124000,101.779,10,101.464,6,Singapore
3,BEA,117.111133,101.891,15,102.666,15,Singapore
4,BOR,110.747818,102.561,11,102.277,14,Singapore


In [7]:
feature_cols = ['fp2_avg_lap', 'fp2_best_lap', 'fp2_total_laps', 'FastestQualiLap', 'QualiPosition'] 
X_singapore = df_singapore[feature_cols]

X_singapore_scaled = scaler.transform(X_singapore) 
proba_podium = model.predict_proba(X_singapore_scaled)[:, 1]

results = df_singapore.copy()
results["podium_probability"] = proba_podium

results_sorted = results.sort_values(by="podium_probability", ascending=False)

predicted_podium = results_sorted[["Driver", "podium_probability"]]

In [8]:
predicted_podium

,Driver,podium_probability
19,VER,0.965896
14,PIA,0.872763
12,NOR,0.871562
15,RUS,0.771569
11,LEC,0.659382
2,ANT,0.517183
10,LAW,0.409543
16,SAI,0.299297
7,HAD,0.174074
18,TSU,0.094905


In [9]:
# Singapore GP 2025
predicted_podium.head(3)

,Driver,podium_probability
19,VER,0.965896
14,PIA,0.872763
12,NOR,0.871562
